## 概念介绍

token: 最小语义单元

tokenizer: 分词器，每个模型都会有自己的tokenizer，将输入文本划分为token

tokenization: 分词的过程

流程

输入文本， token， 模型推理， 输出   

## pipeline

简化上述全部中间过程

transformer中会预设一些类型的模型（如下），当你选择一个类型时，它会去下载一个该类型的模型（具体下载哪个不清楚），然后使用该模型。

```python
from transformers import pipeline

# # # 实体命名识别
# ner = pipeline("ner", grouped_entities=True)
# print(ner("My name is Sylvain and I work at Hugging Face in Brooklyn."))

# # # 掩码填充
# fill_mask = pipeline("fill-mask")
# print(fill_mask("The cat is <mask> on the mat."))

# # # 翻译
# translator = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en")
# print(translator("Ce cours est produit par Hugging Face."))

# # # 文章总结
# summarizer = pipeline("summarization")
# print(summarizer("xxxxxxxxxxxxxxxxxx"))

# # # 文本生成
# generator = pipeline("text-generation")
# print(generator("In this course, we will teach you how to"))
# # # 指定hugging face Hub网站中任意模型
# generator = pipeline("text-generation", model="distilgpt2")
# print(generator("In this course, we will teach you how to", max_length=30, num_return_sequences=2))

# # # 问题回答
# question_answerer = pipeline("question-answering")
# print(question_answerer(question="Where do I work?", context="My name is Sylvain and I work at Hugging Face in Brooklyn"))

# # 情绪分析
classifier = pipeline("sentiment-analysis")   # # （该库只能输入英文。）
print(classifier(["I've been waiting for a HuggingFace course my whole life.", "I hate this so much!"]))

# # # 零样本分类zero-shot-classification
# classifier = pipeline("zero-shot-classification")
# print(classifier("This is a course about the Transformers library", candidate_labels=["education", "politics", "business"]))

# # # 获取文本的向量表示
# feature_extraction = pipeline("feature-extraction") 
# print(feature_extraction("i am a studet"))
```

In [2]:
from transformers import pipeline

# 情绪分析
pipe = pipeline("sentiment-analysis")   # （该模型只能处理英文。）
print(pipe(["This course is amazing.", "I hate this so much!"]))

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998829364776611}, {'label': 'NEGATIVE', 'score': 0.9994558691978455}]


上面使用的是 sentiment-analysis, 它下载了 distilbert/distilbert-base-uncased-finetuned-sst-2-english 这个模型
另外也可以使用pipe下载指定的模型

In [3]:
from transformers import pipeline

pipe = pipeline("text-classification", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")
print(pipe(["This course is amazing.", "I hate this so much!"]))

[{'label': 'POSITIVE', 'score': 0.9998829364776611}, {'label': 'NEGATIVE', 'score': 0.9994558691978455}]


## 常规流程
### 转为token

将输入的原始文本raw text 转换为tokens（单词words、子单词subwords、符号symbols 都算）
将每个tokens 与一个整数做映射，得到Input IDs；
同时也会添加可能对模型有用的额外输入，比如起始字符、隔断字符等；
同一个单词在不同模型的tokenizer下得到的Input IDs数值可能不同，而Input IDs数值必须要与当前使用的预训练模型保持通用。为此，AutoModel 和AutoTokenizer 请共用一个模型检查点checkpoint

padding：填充，确保所有样本在序列长度上是一致的，如果样本的长度小于既定长度，将用填充字符进行补齐；
truncation：截断，确保所有样本序列长度不超过既定长度，若超过，进行截断处理；
max_length ：既定长度，默认为True，不同模型的既定长度可能不一样，也可以用数值指定；
return_tensors：指定返回的张量类型，“pt”：PyTorch，“tf”：TensorFlow ，“np”：NumPy；

不同标志器编码输出的张量结构也存在一定差异，这个是包含两个键的字典：

input_ids：每一行代表一个编码后的句子，0 表示填充字符
attention_mask：1 表示要注意该位标记，0 表示不注意该位标记（即忽略该位），主要用于self-attention层

input_ids：代表编码后的输出IDs；
token_type_ids：是在BERT模型中用于表示不同句子的标识符，标识哪些部分是前句子，哪些部分是后句子；
attention_mask：主要用于self-attention层。1 表示要注意相应位置的标记，0 表示忽略相应位置的标记。

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# # 加载标志器，模型
checkpoint = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

raw_text = ["This course is amazing.", "I hate this so much!"]

tokens = tokenizer(raw_text, padding=True, truncation=True, max_length = 10, return_tensors="pt")
print(tokens)


{'input_ids': tensor([[ 101, 2023, 2607, 2003, 6429, 1012,  102,    0],
        [ 101, 1045, 5223, 2023, 2061, 2172,  999,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1]])}


In [15]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# # 加载标志器，模型
checkpoint = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

raw_text = ["This course is amazing.", "I hate this so much!"]


tokenized_sequence  = tokenizer.tokenize(raw_text)
tokens = tokenizer(raw_text, padding=True, truncation=True, max_length = 10, return_tensors="pt")

print(tokenized_sequence)
print(tokens)


{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['this', 'course', 'is', 'amazing', '.', 'i', 'hate', 'this', 'so', 'much', '!']
{'input_ids': tensor([[ 101, 2023, 2607, 2003, 6429, 1012,  102,    0],
        [ 101, 1045, 5223, 2023, 2061, 2172,  999,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1]])}


### 模型预测
和标志器tokenizer 一样，Transformers 也提供了一个AutoModel 类和from_pretrained() 方法——在使用该模型检查点checkpoint 时，自动下载、加载、使用该模型。
而在具体任务中，模型需要根据不同的任务需求，选择加载不同的模型头model head。模型头是附加组件，通常由一层或几层组成，用于将预测转换为特定任务的输出，下面是一个非详尽的模型头列表：（下表里的 -xxx 表示省略，请与前文保存一致）

*ForCausalLM：因果语言模型，用于文本生成、对话生成，例如GPT2ForCausalLM，XLNetForCausalLM；
*ForMaskedLM：掩码模型，用于理解语言的上下文和语境，例如BERTForMaskedLM、RobertaForMaskedLM；
*ForMultipleChoice：多项选择模型，常见于问答系统，例如BertForMultipleChoice、XLNetForMultipleChoice；
*ForQuestionAnswering：问答模型，例如BertFor-xxx、DistilBertFor-xxx、XLNetForQ-xxx；
*ForSequenceClassification：序列分类模型，常见于文本分类、情感分析，例如BertFor-xxx、DistilBertFor-xxx、XLNetFor-xxx；
*ForTokenClassification：标记分类模型，常见于命名实体识别，例如BertFor-xxx、DistilBertFor-xxx、XLNetFor-xxx；
*Model：用于检索模型架构的隐藏状态
其他
对于我们的示例，我们需要模型具有序列分类的能力（能够将句子分类为正例或负例），因此我们实际不会使用AutoModel 用于初始化，而是会根据具体需求使用AutoModelForSequenceClassification 这个模型头；

In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# # 加载标志器，模型
checkpoint = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

raw_text = ["This course is amazing.", "I hate this so much!"]

tokens = tokenizer(raw_text, padding=True, truncation=True, max_length = 10, return_tensors="pt")
print(tokens)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**tokens)
print(outputs.logits)

{'input_ids': tensor([[ 101, 2023, 2607, 2003, 6429, 1012,  102,    0],
        [ 101, 1045, 5223, 2023, 2061, 2172,  999,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1]])}
tensor([[-4.3690,  4.6833],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>)


### 后处理
我们从模型中获得的输出值本身并不一定有意义，例如上例中的输出，它并不是概率，而是logits，要转换为概率，还需要经过SoftMax层，这样就可以看到输出的概率；
再配合每个位置对应的标签：
第一句：有0.0117%的概率是NEGATIVE，只有99.988%的概率是POSITIVE
第二句：有99.946%的概率是NEGATIVE，有0.0544%的概率是POSITIVE

In [10]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

model.config.id2label

tensor([[1.1711e-04, 9.9988e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward0>)


{0: 'NEGATIVE', 1: 'POSITIVE'}

### 保存模型
还是以BERT 模型为例，使用配置参数BertConfig 创建模型，以随机数权重进行初始化；
可以输出BERT 模型的配置参数，这些参数包含许多用于构建模型的基本属性信息；

上述创建模型的方式是使用随机权重进行的初始化，相当于创建了一个全新的、未训练的模型。该状态下的模型也可以使用，但会输出乱码，需要注入数据 + 需要时间训练——不必重复造轮子，请使用共享的、已经训练好的预训练模型吧！

保存模型
保存模型就像加载模型一样简单，保存模型用：
model.save_pretrained("model_path")

将会有两类文件保存在model_path目录下，这两类文件缺一不可；
配置文件：保存模型架构，权重文件：保存模型权重（权重文件可能有多个）。

config.json          # # 配置文件：保存构建模型架构所需的所有属性
pytorch_model.bin    # # 权重文件：保存模型所有的权重数值，可能有多个.bin文件

In [ ]:
from transformers import BertConfig, BertModel

# # 读取配置参数
config = BertConfig()
print(config)
# # 配置参数构建模型
model = BertModel(config)   # # 模型是随机初始化的！

# BertConfig {
#   [...]
#   "hidden_size": 768,              # # 隐藏层中神经元的数量
#   "intermediate_size": 3072,       # # FNN前馈网络中间层的维度
#   "max_position_embeddings": 512,  # # 最大序列长度，也是位置矩阵的长度
#   "num_attention_heads": 12,       # # 注意力头的数量
#   "num_hidden_layers": 12,         # # 隐藏层的数量
#   [...]
# }
